In [1]:
import csv
import random
from datetime import datetime, timedelta

def generate_random_time(start_time, end_time, precision="second"):
    """
    Generate a random time between start_time and end_time with the specified precision.
    """
    if precision == "second":
        return start_time + timedelta(seconds=random.uniform(0, (end_time - start_time).total_seconds()))
    elif precision == "millisecond":
        return start_time + timedelta(milliseconds=random.uniform(0, (end_time - start_time).total_seconds() * 1000))
    else:
        raise ValueError("Invalid precision. Use 'second' or 'millisecond'.")

def generate_data_points(num_points, precision="second"):
    """
    Generate random data points for vehicles entering the network and passing through RSU1.
    """
    start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start time for simulation
    end_time = datetime(2023, 1, 1, 23, 59, 59)  # End time for simulation
    vehicle_type = ['Motorcycle', 'Car', 'Bus', 'Truck']
    vehicle_type_int = {
        'Motorcycle': 0,
        'Car': 1,
        'Bus': 2,
        'Truck': 3
    }
    probability_of_vehicles = [0.45, 0.2, 0.15, 0.20]
    lanes = [1, 2, 3, 4]
    processing_time_matrix = [
        [10, 15, 20, 25],
        [20, 35, 50, 90],
        [35, 70, 80, 90],
        [120, 140, 90, 75]
    ]

    data_points = []
    

    for vehicle_id in range(num_points):
        # vehicle_type = random.choices(vehicle_type, weights = probability_of_vehicles)[0]
        vehicle_type = random.choices(['Motorcycle', 'Car', 'Bus', 'Truck'], weights=[0.45, 0.2, 0.15, 0.20])[0]
        passing_time_rsu1 = generate_random_time(start_time, end_time, precision)
        passing_time_rsu2 = passing_time_rsu1 + timedelta(seconds=30) #generate_random_time(entry_time, end_time, precision)
        initial_lane = random.choice([0, 1, 2, 3])
        final_lane = initial_lane #same lane because fuzzy logic not implemented
        service_time = processing_time_matrix[vehicle_type_int[vehicle_type]][initial_lane-1] #time to switch lane and cross tollgate 
        exit_time = passing_time_rsu2 + timedelta(seconds=service_time)
        
        data_points.append({
            "Vehicle_ID": vehicle_id+1,
            "Vehicle_Type": vehicle_type,
            "Vehicle_Type(int)": vehicle_type_int[vehicle_type],
            "Initial_Lane": initial_lane,
            "Final_Lane": final_lane,
            "RSU1_Passing_Time": passing_time_rsu1.strftime("%H:%M:%S.%f")[:-3] if precision == "millisecond" else passing_time_rsu1.strftime("%H:%M:%S"),
            "RSU2_Passing_Time": passing_time_rsu2.strftime("%H:%M:%S.%f")[:-3] if precision == "millisecond" else passing_time_rsu2.strftime("%H:%M:%S"),
            "Service_Time": service_time,
            "Exit_Time": exit_time.strftime("%H:%M:%S.%f")[:-3] if precision == "millisecond" else exit_time.strftime("%H:%M:%S")
        })

    return data_points

def save_to_csv(data_points, file_path):
    """
    Save data points to a CSV file.
    """
    with open(file_path, mode='w', newline='') as file:
        fieldnames = ["Vehicle_ID", "Vehicle_Type", "Vehicle_Type(int)", "Initial_Lane", "Final_Lane", "RSU1_Passing_Time", "RSU2_Passing_Time", "Exit_Time", "Service_Time"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for data_point in data_points:
            writer.writerow(data_point)

if __name__ == "__main__":
    num_data_points = 1000  # Change this value based on how many data points you want
    precision = "millisecond"  # Change this to "second" for second precision
    generated_data = generate_data_points(num_data_points, precision)
    save_to_csv(generated_data, "vehicle_data.csv")
